In [1]:
import pandas as pd
import numpy as np

In [ ]:
newspaper_wide=pd.read_csv("/home/jian/Projects/Big_Lots//Ongoing/Cmobined Newspaper Circular by zip/BL_newspaper wide of date by zip_JL_20180507.csv",dtype=str)
newspaper_wide['zip_cd']=newspaper_wide['zip_cd'].replace("00nan","NULL")
store_circ=pd.read_excel("/Users/Jian/Desktop/Media Storm/Big Lots/Ongoing/Cmobined Newspaper Circular by zip/BL_combined store circular data_JL_20180412.xlsx",dtype=str)
newspaper_wide_num_cols=newspaper_wide.columns.tolist()
newspaper_wide_num_cols.remove('zip_cd')
for col in newspaper_wide_num_cols:
    newspaper_wide[col]=newspaper_wide[col].astype(float)

In [ ]:
store_circ['Ship Volume (Ship)']=store_circ['Ship Volume (Ship)'].astype(float)
store_circ=store_circ.groupby(['Zip_5','Date'])['Ship Volume (Ship)'].sum().to_frame()
store_circ.reset_index(inplace=True)
store_circ.columns=['zip_cd','Date','Volumne']
store_circ['Date']=store_circ['Date'].apply(lambda x: "store_circ_"+x[0:10])
store_circ_wide=store_circ.pivot(index="zip_cd",columns="Date",values="Volumne").reset_index()

In [ ]:
newspaper_wide.columns=['zip_cd']+["col_"+x[(len(x)-10):]+"_"+x[0:9] for x in newspaper_wide.columns.tolist()[1:]]
store_circ_wide.columns=['zip_cd']+["col_"+x[(len(x)-10):]+"_"+x[0:10] for x in store_circ_wide.columns.tolist()[1:]]
new_cols=list(set(newspaper_wide.columns.tolist()+store_circ_wide.columns.tolist()))
new_cols.remove("zip_cd")
new_columnes=["zip_cd"]+sorted(new_cols)

In [ ]:
variable_df=pd.merge(newspaper_wide,store_circ_wide,on="zip_cd",how="outer")[new_columnes]

In [ ]:
zips_P_S_store=pd.read_excel("/Users/Jian/Desktop/Media Storm/Big Lots/New TA/zips in new ta/BL_Zips in new TA (TA level)_JL_20180330.xlsx",dtype=str)
zips_P_S=zips_P_S_store.groupby(['zip_cd','revenue_flag'])['location_id'].apply(set).to_frame()
zips_P_S.reset_index(inplace=True)
zips_P_S['location_id']=zips_P_S['location_id'].apply(lambda x:list(x))
zips_P_S['store_1_in_zip']=zips_P_S['location_id'].apply(lambda x: x[0])
zips_P_S['store_2_in_zip']=np.nan
for i in range(len(zips_P_S)):
    if len(zips_P_S['location_id'][i])==2:
        zips_P_S['store_2_in_zip'][i]=zips_P_S['location_id'][i][1]
        print(zips_P_S['zip_cd'][i])
    elif len(zips_P_S['location_id'][i])==1:
        zips_P_S['store_2_in_zip'][i]='nan'
    else:
        zips_P_S['store_2_in_zip'][i]="Check_Error"
del zips_P_S['location_id']
zips_P_S['store_1_in_zip']=zips_P_S['store_1_in_zip'].replace("nan",np.nan)
zips_P_S['store_2_in_zip']=zips_P_S['store_2_in_zip'].replace("nan",np.nan)

In [ ]:
demographic=pd.read_csv("/Users/Jian/Desktop/Media Storm/Docs/Household and Population/2016/output/Demo_Dataset.csv",dtype=str)
demographic['ZIP_CODE']=demographic['ZIP_CODE'].apply(lambda x: x.zfill(5))
demographic=demographic[["ZIP_CODE","HH15","Estimate; Female: - 25 to 29 years","Estimate; Female: - 30 to 34 years",
                         "Estimate; Female: - 35 to 39 years","Estimate; Female: - 40 to 44 years",
                         "Estimate; Female: - 45 to 49 years","Estimate; Female: - 50 to 54 years"]]
num_col_demo=["HH15","Estimate; Female: - 25 to 29 years","Estimate; Female: - 30 to 34 years",
                         "Estimate; Female: - 35 to 39 years","Estimate; Female: - 40 to 44 years",
                         "Estimate; Female: - 45 to 49 years","Estimate; Female: - 50 to 54 years"]
for col in num_col_demo:
    demographic[col]=demographic[col].astype(float)
demographic['Female_25_54']=demographic['Estimate; Female: - 25 to 29 years']+demographic['Estimate; Female: - 30 to 34 years']+\
demographic['Estimate; Female: - 35 to 39 years']+demographic['Estimate; Female: - 40 to 44 years']+\
demographic['Estimate; Female: - 45 to 49 years']+demographic['Estimate; Female: - 50 to 54 years']
demographic=demographic[['ZIP_CODE','HH15','Female_25_54']]
demographic.columns=['zip_cd','HH15','Female_25_54']
demographic['HH15']=demographic['HH15'].replace(-1,np.nan)

In [ ]:
Loyal_member=pd.read_csv("/Users/Jian/Desktop/Media Storm/Big Lots/Ongoing/Cmobined Newspaper Circular by zip/Idsbyzipcode.csv",dtype=str)
Loyal_member.columns=['zip_cd','loyal_members']
Loyal_member['zip_cd']=Loyal_member['zip_cd'].apply(lambda x: x.zfill(5))
Loyal_member['loyal_members']=Loyal_member['loyal_members'].astype(float)

In [ ]:
result=pd.merge(zips_P_S,demographic,on='zip_cd',how="outer")
result=pd.merge(result,Loyal_member,on="zip_cd",how="left")
result=pd.merge(result,variable_df,on="zip_cd",how="outer")

In [ ]:
result=result.sort_values("zip_cd")

In [ ]:
result.to_excel("/Users/Jian/Desktop/Media Storm/Big Lots/Ongoing/Cmobined Newspaper Circular by zip/output/BL_circula data_JL_20180507.xlsx",index=False)